In [1]:
from dotenv import load_dotenv

load_dotenv('../.env')

from langchain.llms import OpenAI
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))


/Users/venkat/opt/anaconda3/envs/llm/lib/python3.12/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.25) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(




Day 1:
Warm-up: Light jog for 5 minutes

Circuit 1:
1. 30 second sprints
2. 1 minute easy jog
3. 30 second high knees
4. 1 minute easy jog
Repeat circuit 3 times

Circuit 2:
1. 1 minute mountain climbers
2. 1 minute jump squats
3. 1 minute burpees
4. 1 minute rest
Repeat circuit 3 times

Cool down: 10 minute brisk walk

Day 2:
Warm-up: 5 minute jog

Circuit 1:
1. 5 minute fast paced run
2. 1 minute rest
3. 3 minute run uphill
4. 1 minute rest
Repeat circuit 3 times

Circuit 2:
1. 20 minute bike ride
2. 1 minute rest
3. 10 minute run
4. 1 minute rest
Repeat circuit 2 times

Cool down: 5 minute slow jog

Day 3:
Warm-up: 5 minute jog

Circuit 1:
1. 1 minute jogging in place
2. 1 minute high knees
3. 1 minute jump rope
4


In [2]:
# LLM chain = PromptTemplate + a model + an optional output parser. 

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"], 
    template="What is the indian company that makes {product} ?"
    )

chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("mobile phones"))



Some popular Indian companies that produce mobile phones are:
1. Xiaomi
2. Samsung
3. Realme
4. Oppo
5. Vivo
6. Micromax
7. Lava
8. Karbonn
9. Intex
10. Jio Phone (Reliance Jio)


In [5]:
# prompt memory = model + conversation chain 

from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conv = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
    )

conv.predict(input="I have 20 ruppees with me.")
conv.predict(input="The shop has water bottles 10 ruppees each")
conv.predict(input="How many water bottles can i buy ?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: I have 20 ruppees with me.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I have 20 ruppees with me.
AI:  That's great! Did you know that the ruppee is the official currency of India? It was first introduced in 1540 by the Afghan ruler Sher Shah Suri. Today, it is used in India, Nepal, and Bhutan. 20 ruppees may not seem like a lot, but it can buy you a cup of chai tea or a plate of samosas in 

" With 20 ruppees, you can buy two water bottles at the price of 10 ruppees each. However, if you want to save some ruppees, you can also refill your water bottle at a public water station for free. It's a more environmentally friendly option too!"

# Retrieval QA from deeplake

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import os 

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

lines = []
with open("planet.txt", "r") as f:
    lines.append(f.readline())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(lines)

my_activeloop_org_id = "test-venkat" 
my_activeloop_dataset_name = "planets"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# add documents to our Deep Lake dataset
db.add_documents(docs)


Your Deep Lake dataset has been successfully created!


Creating 1 embeddings in 1 batches of size 1:: 100%|██████████| 1/1 [00:42<00:00, 42.29s/it]

Dataset(path='hub://test-venkat/planets', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (1, 1)      str     None   
 metadata     json      (1, 1)      str     None   
 embedding  embedding  (1, 1536)  float32   None   
    id        text      (1, 1)      str     None   


['dd9c20f6-e71e-11ee-8f7d-f21898795428']

In [14]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [15]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [16]:
response = agent.run("what is Celestoria Prime?")
print(response)



> Entering new  chain...
 I should use the Retrieval QA System to find the answer.
Action: Retrieval QA System
Action Input: "What is Celestoria Prime?"
Observation:  Celestoria Prime is a fictitious planet located in the Gemini Cluster on the fringes of the galactic expanse. It is known for its unique features, diverse ecosystems, and a thriving civilization that lives in harmony with the planet's natural wonders.
Thought: I now know the final answer.
Final Answer: Celestoria Prime is a fictitious planet located in the Gemini Cluster.

> Finished chain.
Celestoria Prime is a fictitious planet located in the Gemini Cluster.


# Few shot template
Generalize answers from a list of templates.    

In [22]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# examples 
prefix = """Celestoria Prime, a celestial marvel nestled in the cosmic tapestry, 
    orbits a distant star in the Gemini Cluster, located on the fringes of the galactic 
    expanse. This fictitious planet is a testament to the wonders that can emerge in 
    the vastness of the cosmos, boasting a plethora of unique features, diverse ecosystems, 
    and a civilization that thrives in harmony with the planet's natural splendors. 
    The following are excerpts from conversations with an AI
    assistant. The assistant is known for its humor and wit, providing
    entertaining and amusing responses to users' questions. Here are some
    examples:
    """

suffix = """
User: {query}
AI: """

data = [
    {
        "query": "what are the physical characteristics of Celestoria Prime?",
        "answer": """Celestoria Prime is a rocky terrestrial planet with a diameter approximately 1.5 times that of Earth. Its surface is adorned with sprawling landscapes, majestic mountain ranges, and vast expanses of crystalline lakes that shimmer under the warm embrace of the system's sun, Luminary Helios. The planet's gravity is slightly stronger than Earth's, providing a sense of solidity to its terrain."""
    },
    {
        "query": """Describe the biodiversity of Celestoria Prime""",
        "answer": """Celestoria Prime boasts an astonishing biodiversity, with over 500,000 documented species of flora and fauna. Conservation efforts are integral to Celestrian culture, ensuring the preservation of the planet's unique ecosystems."""
    }
]

promptTemplate = """
User: {query}
AI: {answer}
"""

examplePrompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=promptTemplate
)

few_shot_prompt_template = FewShotPromptTemplate(
    examples=data,
    example_prompt=examplePrompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

# load the model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("Should i move to planet celestoria prime ?")

'Well, that depends on your preferences! If you enjoy breathtaking natural beauty, diverse ecosystems, and a harmonious civilization, then Celestoria Prime might be the perfect place for you. Just make sure to pack your space suit and prepare for a cosmic adventure!'